จาก ep ก่อน ๆ เราจะได้ Training Loop หลัก ๆ เป็นดังนี้ (ไม่รวม Metrics) 

```
    for e in range(epoch):
        for xb, yb in train_dl:
            yhatb = model(xb)
            loss = loss_func(yhatb, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()
```

ถ้าเราต้องการเพิ่มเติม Logic เช่น Early Stopping, Learning Rate Annealing, BatchNorm เราจะต้องแก้โค้ดนี้ แทรกตามบรรทัดต่าง ๆ เช่น ก่อนเริ่มเทรน,่่่ก่อนเริ่ม Epoch, ก่อนอัพเดท Weight, หลังจากจบ 1 Epoch, หลังจากเทรนจบ, etc.

ข้อเสียของการแทรกโค้ดแบบนี้ คือ ทำให้โค้ดใน Loop นี้ก็จะบวมขึ้นเรื่อย ๆ ส่งผลให้มีปัญหาในการ Maintain 

ทางแก้ก็คือ เราจะแทรกโค้ดไว้ก่อนเลยในทุกจุดที่เป็นไปได้ เป็นการ Call Function ภายนอก ที่ตั้งชื่อตาม Event ต่าง ๆ ของ Training Loop เรียกว่า [Callback](https://www.bualabs.com/archives/2238/what-is-callback-function-python-ep-6/) ถ้าใครอยากให้ Execute โค้ด ตรงตำแหน่งไหน ก็ตั้งชื่อฟังก์ชันให้ตรงกับชื่อ Event แล้วพาสมาให้กับ Training Loop 

เราจะได้ Training Loop ใหม่เป็นแบบนี้

```
    cb.begin_fit()
    for e in range(epoch):
        cb.begin_epoch()
        for xb, yb in train_dl:
            cb.begin_batch()
            yhatb = model(xb)
            loss = loss_func(yhatb, yb)
            cb.after_loss()
            loss.backward()
            cb.after_backward()
            optim.step()
            cb.after_step()
            optim.zero_grad()
        cb.after_epoch()
    cb.after_fit()

```

เนื่องจากการเทรนหนึ่งครั้ง อาจจะมีหลาย Callback เราจึงสร้าง cb CallbackHandler ขึ้นมาจัดการรวบรวม และวน Loop Call ไปยัง Callback แต่ละตัวที่ได้รับ ตามลำดับที่กำหนดอีกต่อหนึ่ง

# Credit

* https://course.fast.ai/videos/?lesson=9